In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model
from utils import create_dataset_from_one_image

In [2]:
PARENT_PATH = 'models/f001_w_val_data'
# Model/training hyperparameters
INPUT_SHAPE = (512, 512, 3)
CROPPING_MODE = 'independent_patches'  # (copy-paste a mode from: independent_patches, overlapped_patches, random_patches)
TRAIN_IMAGES_NUMBER = None # required only for 'random_patches' mode
PATCHES_STEP = None  # required only for 'overlapped_patches' mode
INITIAL_FILTERS = 32
BATCH_SIZE = 8
EPOCHS = 150

In [3]:
# Load the initial input and output images and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)

x_initial_train = plt.imread('train/101-INPUT.jpg').astype('float32')[650:-550]
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')[650:-550]
x_initial_valid = plt.imread('validation/201-INPUT.jpg').astype('float32') [500:-50]
y_initial_valid = plt.imread('validation/201-OUTPUT-GT.png')[500:-50]

In [4]:
# Create a dataset with the selected cropping mode
x_train, y_train = create_dataset_from_one_image(x_initial_train, 
                                                 y_initial_train, 
                                                 INPUT_SHAPE[0], 
                                                 mode=CROPPING_MODE, 
                                                 new_images_number=TRAIN_IMAGES_NUMBER, 
                                                 patches_step=PATCHES_STEP)

training_images_number = x_train.shape[0]

Crops creation with mode "independent_patches" has been started.
New dataset shapes:
 - Input data = (160, 512, 512, 3)
 - Target data = (160, 512, 512, 1)


In [5]:
# create a validation dataset
x_valid, y_valid = create_dataset_from_one_image(x_initial_valid, 
                                                 y_initial_valid, 
                                                 INPUT_SHAPE[0], 
                                                 mode='independent_patches')

valid_images_number = x_valid.shape[0]

# select a subset from the validation dataset equal to 33% of the training dataset
# NOTE: If 0.33 * training_images_number > valid_images_number, then an Index Error is raised
val_data_size = training_images_number // 3
perm = np.random.permutation(valid_images_number)
x_valid = x_valid[perm][:val_data_size]
y_valid = y_valid[perm][:val_data_size]

print(x_valid.shape)

Crops creation with mode "independent_patches" has been started.
New dataset shapes:
 - Input data = (187, 512, 512, 3)
 - Target data = (187, 512, 512, 1)
(53, 512, 512, 3)


In [6]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [7]:
BATCH_SIZE = 8

# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [8]:
training_images_number = x_train.shape[0]
steps_per_epoch = training_images_number // BATCH_SIZE

# Define the model and train
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS)
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [9]:
if not os.path.exists(PARENT_PATH):
    os.makedirs(PARENT_PATH)

model_name = f'unet_input{INPUT_SHAPE[0]}_filters{INITIAL_FILTERS}_images{x_train.shape[0]}_{CROPPING_MODE}_optAdam_lr{"{:0.4f}".format(model.optimizer.lr.value().numpy())}_eps{EPOCHS}' #TODO: loss and metric'
model_path = os.path.join(PARENT_PATH, model_name)

tensorboard = TensorBoard(log_dir=os.path.join(PARENT_PATH, f'logs/{model_name}'))

In [10]:
history = model.fit(train_generator, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS,
                    validation_data=(x_valid, y_valid),
                    callbacks=[tensorboard]
                   )

Epoch 1/150
20/20 [==============================] - 17s 612ms/step - loss: 0.5908 - accuracy: 0.6859 - val_loss: 7839.1216 - val_accuracy: 0.7955
Epoch 2/150
20/20 [==============================] - 8s 390ms/step - loss: 0.5283 - accuracy: 0.7477 - val_loss: 794.9669 - val_accuracy: 0.7955
Epoch 3/150
20/20 [==============================] - 8s 389ms/step - loss: 0.4778 - accuracy: 0.7770 - val_loss: 82.3383 - val_accuracy: 0.7831
Epoch 4/150
20/20 [==============================] - 7s 381ms/step - loss: 0.4514 - accuracy: 0.7898 - val_loss: 2.7145 - val_accuracy: 0.5220
Epoch 5/150
20/20 [==============================] - 7s 384ms/step - loss: 0.4076 - accuracy: 0.8210 - val_loss: 1.2711 - val_accuracy: 0.7795
Epoch 6/150
20/20 [==============================] - 7s 378ms/step - loss: 0.3560 - accuracy: 0.8487 - val_loss: 4.0548 - val_accuracy: 0.6913
Epoch 7/150
20/20 [==============================] - 7s 382ms/step - loss: 0.3529 - accuracy: 0.8439 - val_loss: 1.7089 - val_accuracy:

Epoch 58/150
20/20 [==============================] - 8s 395ms/step - loss: 0.1281 - accuracy: 0.9521 - val_loss: 0.7313 - val_accuracy: 0.8166
Epoch 59/150
20/20 [==============================] - 8s 394ms/step - loss: 0.1253 - accuracy: 0.9526 - val_loss: 1.5843 - val_accuracy: 0.6333
Epoch 60/150
20/20 [==============================] - 8s 400ms/step - loss: 0.1442 - accuracy: 0.9435 - val_loss: 0.5319 - val_accuracy: 0.7964
Epoch 61/150
20/20 [==============================] - 8s 398ms/step - loss: 0.1598 - accuracy: 0.9398 - val_loss: 0.7249 - val_accuracy: 0.7419
Epoch 62/150
20/20 [==============================] - 8s 393ms/step - loss: 0.1533 - accuracy: 0.9444 - val_loss: 0.6147 - val_accuracy: 0.8499
Epoch 63/150
20/20 [==============================] - 8s 400ms/step - loss: 0.1555 - accuracy: 0.9430 - val_loss: 0.7210 - val_accuracy: 0.8127
Epoch 64/150
20/20 [==============================] - 8s 394ms/step - loss: 0.1199 - accuracy: 0.9534 - val_loss: 0.4718 - val_accuracy:

Epoch 115/150
20/20 [==============================] - 7s 375ms/step - loss: 0.0723 - accuracy: 0.9729 - val_loss: 0.5780 - val_accuracy: 0.8245
Epoch 116/150
20/20 [==============================] - 7s 374ms/step - loss: 0.0798 - accuracy: 0.9698 - val_loss: 0.8135 - val_accuracy: 0.7505
Epoch 117/150
20/20 [==============================] - 7s 372ms/step - loss: 0.0768 - accuracy: 0.9713 - val_loss: 0.5869 - val_accuracy: 0.8214
Epoch 118/150
20/20 [==============================] - 7s 366ms/step - loss: 0.0691 - accuracy: 0.9739 - val_loss: 0.5475 - val_accuracy: 0.8264
Epoch 119/150
20/20 [==============================] - 7s 370ms/step - loss: 0.0676 - accuracy: 0.9757 - val_loss: 0.6470 - val_accuracy: 0.8121
Epoch 120/150
20/20 [==============================] - 7s 377ms/step - loss: 0.0638 - accuracy: 0.9762 - val_loss: 0.8983 - val_accuracy: 0.7848
Epoch 121/150
20/20 [==============================] - 7s 371ms/step - loss: 0.0634 - accuracy: 0.9763 - val_loss: 0.7864 - val_ac

In [11]:
# save the model and history of the training (loss, accuracy)
model.save(model_path)
np.save(os.path.join(model_path, 'history'), history.history)

INFO:tensorflow:Assets written to: models/f001_w_val_data\unet_input512_filters32_images160_independent_patches_optAdam_lr0.0010_eps150\assets


INFO:tensorflow:Assets written to: models/f001_w_val_data\unet_input512_filters32_images160_independent_patches_optAdam_lr0.0010_eps150\assets
